# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Wednesday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**


# Question 1 (40 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]

(3) Collect all the reviews of the top 1000 most popular software from G2 or Capterra.

(4) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(5) Collect all the information of the 904 narrators in the Densho Digital Repository.


In [ ]:
pip install selenium

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

def search():
    rev = []
    z = 1000
    base_url = "https://www.imdb.com/title/tt15398776/reviews?ref_=tt_urv"
    while len(rev) < z:
        r = requests.get(base_url)
       # print(base_url)
        if r.status_code == 200:
            soup = BeautifulSoup(r.content, "html.parser")
            res = soup.find_all("div", class_="lister-item-content")
            for i in res:
                x = {}
                ra = i.find("div", class_="ipl-ratings-bar")
                if ra:
                    x["Rating"] = ra.text.strip()
                redn = i.find("div", class_="display-name-date")
                if redn:
                    x["Name"] = i.find("span", class_="display-name-link").text
                    x["Date"] = i.find("span", class_="review-date").text

                ret = i.find("a", class_="title")
                if ret:
                    x["Review Title"] = ret.text.strip()
                rev.append(x)
                if len(rev) >= z:
                    break
            # Find the "Load More" button and update the URL to fetch the next page of reviews
            load_more_button = soup.find("div", class_="load-more-data")
            if load_more_button:
                next_page_url = "https://www.imdb.com/title/tt15398776/reviews/_ajax?ref_=undefined&paginationKey=" + load_more_button["data-key"]
                #print(next_page_url)
                base_url = next_page_url
            else:
                break
    return rev

def main():
    a = search()
    df = pd.DataFrame(a)
    df.to_csv("search.csv", index=False)
    print(df.to_string(index=False))

if __name__ == "__main__":
    main()


Rating                             Name              Date                                                                                                                                                                Review Title
  8/10                  Jeremy_Urquhart      20 July 2023                                                                                                                       A challenging watch to be sure, but a worthwhile one.
  9/10                  Orlando_Gardner      19 July 2023                                                                                                                                                       Murphy is exceptional
  7/10                      Bonobo13579      21 July 2023                                                                                                                                                      Quality but exhausting
 10/10                         MrDHWong      20 July 2023                       

# Question 2 (30 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [ ]:
a=search()
df=pd.DataFrame(a)
df=df['Review Title']
df.to_csv("review.csv",index=False)
print(df.to_string(index=False))

In [5]:
import pandas as pd
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
sw=set(stopwords.words('english'))
pw=PorterStemmer()
wl=WordNetLemmatizer()

def clean(x):
  x=re.sub(r'[^a-zA-Z\s/]','',x)
  x=re.sub(r'\d+','',x)
  x=x.lower()
  w=nltk.word_tokenize(x)
  w=[i for i in w if i not in sw]
  w=[pw.stem(i) for i in w]
  w=[wl.lemmatize(i) for i in w]
  ct=' '.join(w)

  return ct

def main():
  df = pd.read_csv("search.csv")
  df["Review Title"] = df["Review Title"].apply(clean)
  df.to_csv("clean.csv", index=False)
  print(df.to_string(index=False))


if __name__== "__main__":
  main()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Rating                             Name              Date                                                                                                         Review Title
  8/10                  Jeremy_Urquhart      20 July 2023                                                                                    challeng watch sure worthwhil one
  9/10                  Orlando_Gardner      19 July 2023                                                                                                        murphi except
  7/10                      Bonobo13579      21 July 2023                                                                                                      qualiti exhaust
 10/10                         MrDHWong      20 July 2023                                             brilliantli layer examin man throughout incred accomplish fundament flaw
 10/10                    and_mikkelsen      20 July 2023                                                               nolan

# Question 3 (30 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [6]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import spacy

nltk.download('averaged_perceptron_tagger')

nlp = spacy.load("en_core_web_sm")

def pos(text):
    sent = word_tokenize(text)
    nc, vc, adjc, advc = 0, 0, 0, 0
    for word, tag in pos_tag(sent):
        if tag.startswith('N'):
            nc += 1
        elif tag.startswith('V'):
            vc += 1
        elif tag.startswith('J'):
            adjc += 1
        elif tag.startswith('R'):
            advc += 1
    return nc, vc, adjc, advc

def cpd(text):
    doc = nlp(text)
    for i in doc.sents:
        print("Sentence:", i.text)
        print("Dependency Parsing Tree:")
        for j in i:
            print(j.text, "-->", j.dep_, "-->", j.head.text)
        print()

def ner(text):
    doc = nlp(text)
    pc, oc, lc, prc, dc = 0, 0, 0, 0, 0
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            pc += 1
        elif ent.label_ == "ORG":
            oc += 1
        elif ent.label_ == "LOC":
            lc += 1
        elif ent.label_ == "PRODUCT":
            prc += 1
        elif ent.label_ == "DATE":
            dc += 1
    return pc, oc, lc, prc, dc

def main():
  df = pd.read_csv("clean.csv")
  df.to_csv("clean.csv")
  cs = ''.join(df['Review Title'].astype(str))

  nc,vc,adjc,advc= pos(cs)
  a1 = {"Parts of Speech": ["Noun", "Verb", "Adjective", "Adverb"],"Count": [nc,vc,adjc,advc]}
  df1=pd.DataFrame(a1)


  pc,oc,lc,prc,dc=ner(cs)

  print("Named Entity Recognition:")
  a3= {"Entity": ["Person", "Organization", "Location", "Product", "Date"],"Count": [pc,oc,lc,prc,dc]}
  df2=pd.DataFrame(a3)
  print("Parts of Speech (POS) Tagging:")
  print(df1)
  print()

  print("Constituency and Dependency Parsing:")
  ct=df['Review Title'].iloc[0]
  cpd(ct)
  print()

  print("Named Entity Recognition:")
  print(df2)

if __name__ == "__main__":
    main()


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Named Entity Recognition:
Parts of Speech (POS) Tagging:
  Parts of Speech  Count
0            Noun   1615
1            Verb    271
2       Adjective    593
3          Adverb    114

Constituency and Dependency Parsing:
Sentence: challeng watch sure worthwhil one
Dependency Parsing Tree:
challeng --> nsubj --> watch
watch --> ROOT --> watch
sure --> advmod --> worthwhil
worthwhil --> ccomp --> watch
one --> dobj --> worthwhil


Named Entity Recognition:
         Entity  Count
0        Person     79
1  Organization     36
2      Location      1
3       Product      2
4          Date      7


# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

In [ ]:
"""In this assignment i have learned a lot about the ""